In [ ]:
# テキストコーパスをチャンクに分割 - コードに適した分割方法を使用
with open('code_gate.py', 'r', encoding='utf-8') as f:
    text = f.read()

from langchain.text_splitter import RecursiveCharacterTextSplitter, Language

# コードに特化したスプリッターを使用（Pythonコードの場合）
text_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON,
    chunk_size=500,  # コード理解に適したサイズ
    chunk_overlap=50  # 適度なオーバーラップで文脈を保持
)
texts = text_splitter.split_text(text)

In [ ]:
# パッセージのベクトル化 - コード特化型のエンベディングモデルを使用
from langchain_huggingface import HuggingFaceEmbeddings

# コードに特化したエンベディングモデルを使用
embeddings = HuggingFaceEmbeddings(
#    model_name='Alibaba-NLP/gte-large-code',  # コード特化型エンベディングモデル　★存在しないモデル
    model_name='BAAI/bge-small-en-v1.5',  # 軽量で高性能な汎用エンベディングモデル
#    model_name='microsoft/codebert-base',  # コード用に事前学習されたモデル
#    model_name='nomic-ai/nomic-embed-text-v1',  # 高性能な汎用エンベディング
    model_kwargs={'device': 'cpu'}
)

In [ ]:
from langchain_community.vectorstores import FAISS

# データベースの保存
db = FAISS.from_texts(texts, embeddings)
db.save_local('code.db')

In [ ]:
from langchain_community.vectorstores import FAISS

# 保存したデータベースの読み込み
db = FAISS.load_local('code.db', embeddings, allow_dangerous_deserialization=True)

In [ ]:
# Retrieverの設定 - コード検索向けのパラメータ調整
retriever = db.as_retriever(
    search_type="mmr",  # コードの多様な部分を取得するためにMMRを使用
    search_kwargs={
        "k": 4,  # 取得するドキュメント数
        "fetch_k": 15,  # MMR探索用の初期取得数 
        "lambda_mult": 0.6  # 関連性と多様性のバランス
    }
)

In [ ]:
# モデルの準備
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# DeepSeek Coderモデルを使用
model_id = "deepseek-ai/deepseek-coder-1.3b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto"  # 利用可能なGPUに自動的に配置
).eval()

# DeepSeek Coderに適したパイプライン設定
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,  # より詳細な回答を可能に
    do_sample=True,
    temperature=0.2,  # 低温度で一貫性を向上
    top_p=0.92,  # 確率の高い選択肢に集中
    repetition_penalty=1.1,  # DeepSeekは繰り返しが少ないのでペナルティ軽減
)

In [ ]:
# DeepSeek Coderに適したプロンプトテンプレート
# DeepSeekモデルの推奨プロンプト形式に調整
template = """<问题>
以下のPythonコードを参照して、質問に答えてください:

{context}

質問: {question}
</问题>

<答案>
"""

from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"],
    template_format="f-string"
)

from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFacePipeline

qa = RetrievalQA.from_chain_type(
    llm=HuggingFacePipeline(pipeline=pipe),
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
    verbose=True,
)

In [ ]:
# 実行例

def clean_response(response):
    """DeepSeekの応答から必要な部分を抽出し、整形する"""
    result = response['result'].strip()
    # DeepSeekモデル特有の終了トークンがあれば削除
    if "</答案>" in result:
        result = result.split("</答案>")[0].strip()
    return result

# テスト実行と評価
questions = [
    "どんなプログラミング言語で書かれていますか？",
    "定義されている関数を全てリストアップして下さい",
    "AND関数は何回呼ばれますか？",
    "このプログラムがやっていることは何ですか？"
]

for q in questions:
    print(f"質問: {q}")
    ans = qa.invoke(q)
    cleaned_ans = clean_response(ans)
    print(f"回答: {cleaned_ans}")
    print("\n参照されたドキュメント:")
    for i, doc in enumerate(ans['source_documents'][:2]):  # 最初の2つだけ表示
        print(f"ドキュメント {i+1}: {doc.page_content[:100]}...")
    print("-" * 50)

# 精度評価関数
def evaluate_answers(questions, expected_answers, model_responses):
    """回答の精度を簡易評価する関数"""
    from sklearn.metrics.pairwise import cosine_similarity
    
    # 回答をベクトル化して類似度を計算
    def get_similarity(ans1, ans2):
        # 簡易的なベクトル化と類似度計算
        # 実際のシステムではより高度な方法を使用すべき
        return cosine_similarity([ans1], [ans2])[0][0]
    
    results = []
    for q, exp, res in zip(questions, expected_answers, model_responses):
        sim = get_similarity(exp, res)
        results.append({
            "question": q,
            "similarity": sim,
            "pass": sim > 0.7  # 70%以上の類似度を合格とする
        })
    
    return results
